Below is code written by Victor/Lyudmilla for CHM673. The following code does all the work for setting up a HF calculation, particularly the integrals. It also sets up a crude scf cycle. Read through the code and try to understand how the main idea of HF is represented  computationally, and then do the 2 questions at the end. 

## Hartree-Fock (Self-Consistent-Field) procedure

This notebook demonstrates how the Hartree-Fock procedure works, on the example of a diatomic molecule $He^+H$ in the minimal basis set.  

Despite a seemingly very simple moelcular system, all essential steps of the Hartee-Fock model are included.

So, we are up for solving the Roothaan's equations:
$$ FC = SC\epsilon$$
in which $F$ is the Fock matrix, $S$ is the overlap matrix, $\epsilon$ are molecular orbital eigenvalues, and $C$ are (unknown) expansion coefficients determining (unknown) molecular orbitals:
$$ \psi_i = \sum_{\mu=1}^{K} C_{\mu i} \phi_{\mu} $$

We start with specifying parameters of a chemical system and basis set.

In [9]:
import re, sys, numpy as np, math

# use atomic units throughout
Natom = 2
Nelec = 2

# charges of nuclei: He-H
z1, z2 = 2.0, 1.0
charges = np.array([z1,z2])

# coordinates of nuclei in Bohr
R1 = np.array([0.0, 0.0, 0.0])
R2 = np.array([0.0, 0.0, 0.8])
coord = np.array([R1,R2])

# nuclear repulsion energy
E_nn = 0.0
R12 = np.linalg.norm(R1-R2)
Vnn = z1*z2/R12

np.set_printoptions(formatter={'float': '{: 0.3f}'.format})

# print system info
print("Nuclear charges", charges)
print("Nuclear coordinates")
print(coord)
print(f"Nuclear repulsion energy Vnn = {Vnn} Hartree")

Nuclear charges [ 2.000  1.000]
Nuclear coordinates
[[ 0.000  0.000  0.000]
 [ 0.000  0.000  0.800]]
Nuclear repulsion energy Vnn = 2.5 Hartree


In [10]:
## Define basis set
# Use the simplest model (even simpler than STO-3G): each atom is described 
# with a single Gaussian 

#number of basis functions
Nbasis = 2
# exponents of gaussian functions on atoms 1 (He) and 2 (H)
a = 0.270950*1.8575  # scaled based on STO-2G comparison of He and H
b = 0.270950         # from Szabo & Ostlund -> optimized to fit Slater orbital for H atom

# an array of basis function exponents
BFexp = np.array([a, b])

# normalization for a 3D gaussian function 
def norm(a):
    return np.power(2*a/np.pi, 0.75)
# print(f"Gaussian norm = {norm(a)}")


### Step 1. Computing integrals over basis functions 

Using defined basis functions, we start building the elements of the Roothaan's equation.
Specifically, we need overlap, kinetic energy and electron-nuclear attraction one-electron 
integrals and electron-electron repulsion two-electron integrals:
    
$$ S_{\mu\nu} = \int \phi_{\mu}^*(r)\phi_{\nu}(r)dr $$

$$ T_{\mu\nu} = \int \phi_{\mu}^*(r) \left (-\frac{1}{2} \nabla_r^2  \right ) \phi_{\nu}(r)dr $$

$$ V^{en}_{\mu\nu} = \int \phi_{\mu}^*(r) \left (- \sum_{A}^{N}\frac{Z}{r_A} \right ) \phi_{\nu}(r)dr $$ 

$$ V^{ee}_{\mu\nu pq} = \int \phi_{\mu}^*(r_1)\phi_{\nu}(r_1) \frac{1}{|r_1-r_2|}\phi_{p}^*(r_2)\phi_{q}(r_2) dr_1 dr_2 $$

With gaussian basis functions, all needed integrals are analytical. Speific formulas are 
adapted from Appendix A and B in Szabo & Ostlund.   

First, we define the needed integral functions, and then combine all integrals into matrices 
$S$, $T$, $V^{en}$, $V^{ee}$ (the latter is called in the code as $tei$ - two-electron integrals.

We also can start building the core Hamiltonian matrix:

$$ H_{\mu\nu}^{core} = h_{\mu\nu} = T_{\mu\nu} + V^{en}_{\mu\nu} $$


In [11]:
from scipy import special

# integral functions 

# Ra, Rb are positions of the basis centers
# a and b are basis function exponents

# basis function normalization is included in the integral funtions: 
# see how "norm()" functions are used

# overlap integral 
def overlap(a,b,Ra,Rb):
    RR = np.dot(Rb-Ra,Rb-Ra)
    return norm(a)*norm(b)*np.power(np.pi/(a+b),1.5)*np.exp(-a*b*RR/(a+b))

# kinetic energy integral
def kinetic(a,b,Ra,Rb):
    RR = np.dot(Rb-Ra,Rb-Ra)
    tmp = norm(a)*norm(b)*np.power(np.pi/(a+b),1.5)*np.exp(-a*b*RR/(a+b))
    return tmp*a*b/(a+b)*(3-2*a*b*RR/(a+b))

# error function for Ven and Vee integrals 
def F0(x):
    if abs(x) < 10**(-6):
        return 1.0 - x/3.0  # asymptotic value for small arguments
    else:
        return 0.5*math.sqrt(np.pi/x)*special.erf(math.sqrt(x))
    
# nuclear attraction integrals
# Rz is the coordinate of charge Z 
def V(a,b,Ra,Rb,Rz,Z):
    RR = np.dot(Rb-Ra,Rb-Ra)
    # center of two gaussians
    RP = Ra + b*(Rb-Ra)/(a+b)
    # distance squared between center of two gaussians and position of charge Z
    RRP = np.dot(RP-Rz,RP-Rz)
    tmp = norm(a)*norm(b)*2*np.pi/(a+b)*np.exp(-a*b*RR/(a+b))
    return -Z*tmp*F0((a+b)*RRP)
    
# electron-electron interaction (two-electron integral)
def twoe(a,b,c,d,Ra,Rb,Rc,Rd):
    # distance between first two gaussians
    RRab = np.dot(Ra-Rb,Ra-Rb)
    # ditance between next two gaussians
    RRcd = np.dot(Rc-Rd,Rc-Rd)
    # position of center of the gaussian made of gaussians a and b
    RPab = Ra + b*(Rb-Ra)/(a+b)
    # center of gaussian made of gaussians c and d
    RPcd = Rc + d*(Rd-Rc)/(c+d)
    # distance squared between those two centers (ab and cd)
    RRpq = np.dot(RPab-RPcd,RPab-RPcd)
    tmp = 2*np.pi**2.5/((a+b)*(c+d)*math.sqrt(a+b+c+d))*np.exp(-a*b*RRab/(a+b)-c*d*RRcd/(c+d))
    return norm(a)*norm(b)*norm(c)*norm(d)*tmp*F0((a+b)*(c+d)*RRpq/(a+b+c+d))

In [12]:
# Make one-electron integral matrices
# combine Te and Ven into the core Hamiltonian matrix Hcore

S = np.zeros((Nbasis,Nbasis)) 
Te = np.zeros((Nbasis,Nbasis)) 
Ven = np.zeros((Nbasis,Nbasis))
Hcore = np.zeros((Nbasis,Nbasis)) 

for i in range(Nbasis):
    for j in range(Nbasis):
        S[i,j] = overlap(BFexp[i], BFexp[j],coord[i],coord[j])
print("Overlap S")
print(S)        

for i in range(Nbasis):
    for j in range(Nbasis):
        Te[i,j] = kinetic(BFexp[i], BFexp[j],coord[i],coord[j])
print("Kinetic energy Te")
print(Te)        

for i in range(Nbasis):
    for j in range(Nbasis):
        for z in range(len(charges)):
            Ven[i,j] = Ven[i,j] + V(BFexp[i],BFexp[j],coord[i],coord[j],coord[z],charges[z])
print("Electron-nuclear repulsion Ven")
print(Ven)        

# core Hamiltonian
Hcore = Te + Ven
print("Core Hamiltonian Hcore")
print(Hcore)        


Overlap S
[[ 1.000  0.832]
 [ 0.832  1.000]]
Kinetic energy Te
[[ 0.755  0.407]
 [ 0.407  0.406]]
Electron-nuclear repulsion Ven
[[-3.194 -2.392]
 [-2.392 -2.318]]
Core Hamiltonian Hcore
[[-2.439 -1.985]
 [-1.985 -1.912]]


In [13]:
# prepare two-electron integral matrix tei
# note that G is 4-dimensional!

tei = np.zeros((Nbasis,Nbasis,Nbasis,Nbasis)) 

for i in range(Nbasis):
    for j in range(Nbasis):
        for k in range(Nbasis):
            for l in range(Nbasis):
                tei[i,j,k,l] = twoe(BFexp[i],BFexp[j],BFexp[k],BFexp[l],
                                    coord[i],coord[j],coord[k],coord[l])
print("Two-electron integrals tei")
print(tei)   
#print(tei[0,0,0,0])


Two-electron integrals tei
[[[[ 0.801  0.614]
   [ 0.614  0.623]]

  [[ 0.614  0.486]
   [ 0.486  0.515]]]


 [[[ 0.614  0.486]
   [ 0.486  0.515]]

  [[ 0.623  0.515]
   [ 0.515  0.587]]]]


### Step 2: Orthogonalization of the basis functions

Note that our basis functions are not orthogonal (the ovelap matrix $S$ is not unity). This 
complicates solving the eignevalue problem (Roothaan's equation). To resolve this complexity, 
we will orthogonalize the basis, i.e., find such a (non-unitary) transformation to the basis functions that makes the overlap matrix unity. One way to do it is as following. 
First, the eigenvalues $(\epsilon_S)$ and eigenvectors ($L_S$) of the overlap matrix $S$ are found:
$$ SL_S = L_S\epsilon_S $$

Then, the symmetric orthogonalization matrix (unitary transformation) is constructed as

$$ S^{-1/2} = L_{S} \epsilon_S^{-1/2} L_S^T $$

where $\epsilon_S$ is the diagonal matrix of eigenvalues and $ L_S $ is the matrix of eigenvectors of the overlap matrix $S$. 

This procedure is called *symmetric orthogonalization* - see related discussion in Chapter 3.4.5 of Szabo & Ostlund.

In [14]:
# ortogonalization of the basis: bringing the overlap S matrix to unity

# eigenvalues and eigenvectors of matrix S
S_eigval, S_eigvec = np.linalg.eig(S)

# building the epsilon_S^{-1/2} matrix - placing 1/sqrt(S_eigval) into diagonal
S_eigval_minhalf = np.diag(S_eigval**(-0.5))

# this is the orthogonalization matrix S^{-1/2} 
# @ is a dot-product operator
S_minhalf = S_eigvec @ S_eigval_minhalf @ S_eigvec.T  

print("epsilon_S^{-1/2}")
print(S_eigval_minhalf)

print("\nS^{-1/2}")
print(S_minhalf)


epsilon_S^{-1/2}
[[ 0.739  0.000]
 [ 0.000  2.442]]

S^{-1/2}
[[ 1.591 -0.852]
 [-0.852  1.591]]


### Step 3: Main SCF Loop

In the first cycle, the initial Fock matrix is formed from the core Hamiltonian as a guess (i.e., we set the initial density matrix as 0):

$$ F^{(0)}_{\mu \nu} = H_{\mu\nu}^{core} $$

In the next and all following cycles, the Fock matrix is computed as 

$$ F^{(i)}_{\mu \nu} = H_{\mu\nu}^{core} + \sum_{pq} P_{pq} G_{\mu \nu p q} $$

where $ G_{\mu \nu p q} $ is the difference of Coulomb and exchange integrals and $P_{pq}$ is the density matrix.

$\color{blue}{Note:}$ the core Hamiltonian is constant for a given geometry/basis set and does not need to be updated in the SCF iterations.

Then the Fock matrix is transformed to the orthonormal basis:

$$ F^\prime = ({S}^{-1/2})^T F S^{-1/2} $$ 

in which the Roothaan's equations take the form: 

$$ F^\prime C^\prime = C^\prime \epsilon $$

The transformed Fock matrix $F^\prime$ is diagonalized to produce orbital energies $\epsilon$ 
and eigenvectors (expansion coefficients) $C^\prime$. These eigenvectors are back-transformed into the original (non-orthogonal) atomic basis:

$$ C = S^{-1/2} C^\prime $$

The density matrix is computed from the eigenvectors $C$ as:

$$ P_{\mu \nu} = \sum_{pq}^{Nelec} C_{\mu p} C_{\nu q}  = 2*\sum_{pq}^{Nelec / 2} C_{\mu p} C_{\nu q}$$ 

$\color{blue}{Note:}$ The system is assumed to be a closed shell (spatial orbitals are doubly-occupied, no unpaired electrons).

The current Hartree-Fock energy is obtained as:

$$ E = \frac{1}{2} \sum_{\mu \nu} P_{\mu \nu} (H^{core}_{\mu \nu} + (H^{core}_{\mu \nu} + P_{pq}*G_{\mu \nu p q})) = \frac{1}{2} \sum_{\mu \nu} P_{\mu \nu} (H^{core}_{\mu \nu} + F_{\mu \nu}) $$ 

This finishes the current SCF iteration. 


In [15]:
# handy functions to help with the HF procedure flow

# difference between current and previous density matrices
# computed using Euclidean or L2 norm
def delta_P(P,OLDP):  
    DELTA = 0.0  
    for i in range(Nbasis):  
        for j in range(Nbasis):  
            DELTA = DELTA+((P[i,j]-OLDP[i,j])**2)  
    DELTA = (DELTA/(Nbasis**2))**(0.5)  
    return DELTA

# compute the current energy 
def ene_current(P,Hcore,F):  
    ene = 0.0  
    for mu in range(Nbasis):  
        for nu in range(Nbasis):  
            ene = ene + 0.5*P[mu,nu]*(Hcore[mu,nu] + F[mu,nu])  
    return ene

# compute density matrix from expansion coefficients C 
# note contraction over occupied orbitals (Nelec/2) only
def make_density(C,P,Nelec):  
    OLDP = np.zeros((Nbasis,Nbasis))  
    for mu in range(Nbasis):  
        for nu in range(Nbasis):  
            OLDP[mu,nu] = P[mu,nu]
            P[mu,nu] = 0.0
            for i in range(0,Nelec//2):  # floor division here   
                P[mu,nu] = P[mu,nu] + 2*C[mu,i]*C[nu,i]  
    return P, OLDP

# build Fock matrix from the core Hamiltonian, density matrix and two-electron integrals
# the exchange integrals are halfed to account for same-spin interactions only
def make_fock(Hcore,P):  
    F = np.zeros((Nbasis,Nbasis))  
    for i in range(Nbasis):  
        for j in range(Nbasis):  
            F[i,j] = Hcore[i,j]  
            for k in range(Nbasis):  
                for l in range(Nbasis):  
                    F[i,j] = F[i,j] + P[k,l]*(tei[i,j,k,l]-0.5*tei[i,l,k,j])  
    return F

# converts Fock matrix to orthonormal basis
def make_f_prime(X,F):  
    return np.dot(X.T,np.dot(F,X))


In [16]:
## MAIN SCF CYCLE

# Initialization: density matrix is set to zero initially
P = np.zeros((Nbasis, Nbasis))   

for i in range(1, 11):
    print ("\n*****************************************")
    print ("SCF Cycle ", i)
    
    # build Fock matrix
    F = make_fock(Hcore, P)
    print ("\nFock Matrix :\n", F)
    
    # convert Fock matrix to the orthonormal basis
    Fprime = make_f_prime(S_minhalf,F)
    print ("\nF-prime :\n", Fprime)
    
    # diagonalize Fprime and convert eigenvectors back into original basis
    E,Cprime = np.linalg.eigh(Fprime)
    C = S_minhalf @ Cprime
    print ("\nExpansion coefficients C :\n", C)
    print ("\nOrbital energies :\n", E)
    
    # compute density matrix P from vectors C
    P,OLDP = make_density(C,P,Nelec)
    print ("\nDensity Matrix :\n", P)

    # compute change in density matrix from the previos iteration
    DELTA = delta_P(P,OLDP)
    print (f"\nChange in density: {DELTA:0.8f}")
    
    # compute current Hartree-Fock energy (including the nuclear repulsion energy Vnn)
    EN = ene_current(P,Hcore,F)
    print (f"\nCurrent Energy = {(EN + Vnn):0.8f}")
    
    i += 1

print (f"\n\nEnd of {i} SCF iterations. Final RHF energy = {EN+Vnn:0.8f} Hartree")



*****************************************
SCF Cycle  1

Fock Matrix :
 [[-2.439 -1.985]
 [-1.985 -1.912]]

F-prime :
 [[-2.177 -0.569]
 [-0.569 -1.226]]

Expansion coefficients C :
 [[-1.080  1.445]
 [ 0.098 -1.802]]

Orbital energies :
 [-2.443 -0.960]

Density Matrix :
 [[ 2.333 -0.211]
 [-0.211  0.019]]

Change in density: 1.17595526

Current Energy = -2.38634947

*****************************************
SCF Cycle  2

Fock Matrix :
 [[-1.628 -1.352]
 [-1.352 -1.130]]

F-prime :
 [[-1.273 -0.666]
 [-0.666 -0.375]]

Expansion coefficients C :
 [[-1.004  1.499]
 [ 0.005 -1.804]]

Orbital energies :
 [-1.628 -0.021]

Density Matrix :
 [[ 2.016 -0.010]
 [-0.010  0.000]]

Change in density: 0.21326537

Current Energy = -1.56695463

*****************************************
SCF Cycle  3

Fock Matrix :
 [[-1.638 -1.370]
 [-1.370 -1.152]]

F-prime :
 [[-1.266 -0.681]
 [-0.681 -0.390]]

Expansion coefficients C :
 [[-0.988  1.510]
 [-0.014 -1.804]]

Orbital energies :
 [-1.638 -0.018]

Dens

## Homework assignment 

### Part 1. 
This SCF code is not perfect. Indeed, why are we setting the total number of SCF cycles (= 10) in advance?
Instead, we better do one of the following:

* set an energy change threshold

* set a density change threshold

* set a density as well as energy change thresholds, which is done in most software packages

Modify the HF code to use one of these metrics. Use the following criteria for converegence: 

* energy change threshold $ E_{thresh} = 10^{-8} $ Hartree

* density change threshold $ P_{thresh} = 10^{-6} $ 

### Part 2. 

By running the code several times, determine the equilibrium geometry of HeH+ molecule. 
Alternatively, you can re-organize this code and make a loop in which geometry of the molecule changes gradually. However, if you do so, make sure you understand which information needs to be recomputed when the molecular geometry changes.
